In [ ]:
from omop_alchemy.cdm.registry import CDMModelRegistry
from omop_alchemy.cdm.config import TABLE_LEVEL_CSV, FIELD_LEVEL_CSV
from omop_alchemy.model.vocabulary import (
    Concept,
    Domain,
    Vocabulary,
    Concept_Class,
    Concept_Relationship,
    Concept_Ancestor,
    Concept_Synonym,
    Relationship,
)
registry = CDMModelRegistry(cdm_version="5.4")

registry.load_specs(
    table_csv=TABLE_LEVEL_CSV,
    field_csv=FIELD_LEVEL_CSV,
)

registry.discover_models("omop_alchemy.model")

In [2]:
registry.known_tables()

{'care_site',
 'cdm_source',
 'cohort',
 'cohort_definition',
 'concept',
 'concept_ancestor',
 'concept_class',
 'concept_relationship',
 'concept_synonym',
 'condition_era',
 'condition_occurrence',
 'cost',
 'death',
 'device_exposure',
 'domain',
 'dose_era',
 'drug_era',
 'drug_exposure',
 'drug_strength',
 'episode',
 'episode_event',
 'fact_relationship',
 'location',
 'measurement',
 'metadata',
 'note',
 'note_nlp',
 'observation',
 'observation_period',
 'payer_plan_period',
 'person',
 'procedure_occurrence',
 'provider',
 'relationship',
 'source_to_concept_map',
 'specimen',
 'visit_detail',
 'visit_occurrence',
 'vocabulary'}

In [3]:
report = registry.validate()

In [4]:
print(report.render_text())


📦 care_site
  ⚠️ TABLE_NOT_REGISTERED

📦 cdm_source
  ⚠️ TABLE_NOT_REGISTERED

📦 cohort
  ⚠️ TABLE_NOT_REGISTERED

📦 cohort_definition
  ⚠️ TABLE_NOT_REGISTERED

📦 concept
  ⚠️ TABLE_NOT_REGISTERED

📦 concept_ancestor
  ⚠️ TABLE_NOT_REGISTERED

📦 concept_class
  ⚠️ TABLE_NOT_REGISTERED

📦 concept_relationship
  ⚠️ TABLE_NOT_REGISTERED

📦 concept_synonym
  ⚠️ TABLE_NOT_REGISTERED

📦 condition_era
  ⚠️ TABLE_NOT_REGISTERED

📦 condition_occurrence
  ⚠️ TABLE_NOT_REGISTERED

📦 cost
  ⚠️ TABLE_NOT_REGISTERED

📦 death
  ⚠️ TABLE_NOT_REGISTERED

📦 device_exposure
  ⚠️ TABLE_NOT_REGISTERED

📦 domain
  ⚠️ TABLE_NOT_REGISTERED

📦 dose_era
  ⚠️ TABLE_NOT_REGISTERED

📦 drug_era
  ⚠️ TABLE_NOT_REGISTERED

📦 drug_exposure
  ⚠️ TABLE_NOT_REGISTERED

📦 drug_strength
  ⚠️ TABLE_NOT_REGISTERED

📦 episode
  ⚠️ TABLE_NOT_REGISTERED

📦 episode_event
  ⚠️ TABLE_NOT_REGISTERED

📦 fact_relationship
  ⚠️ TABLE_NOT_REGISTERED

📦 location
  ⚠️ TABLE_NOT_REGISTERED

📦 measurement
  ⚠️ TABLE_NOT_REGISTERED

📦 met

In [ ]:
if not report.is_valid():
    raise ValueError("CDM model validation failed")

In [ ]:
for table, spec in registry._table_specs.items():
    print(f"{table}: {spec.is_required}")